# Intuitional Description

The Bayes factor is the ratio of two **marginal** likelihoods; that is, the likelihoods of two statistical models integrated over the prior probabilities of their parameters.


# Graphical Summary

![fig](./cartoons/fig.svg)

# Key Formula

Given a model selection problem in which one wishes to choose between two models on the basis of observed data $\text{D}$, the plausibility of the two different models $\text{M}_1$ and $\text{M}_2$, parametrised by model parameter vectors $\gamma_1$ and $\gamma_2$, is assessed by the Bayes factor $\text{BF}$ given by:
$$
\text{BF}_{1,2} = \frac{L(\text{D}|\text{M}_1)}{L(\text{D}|\text{M}_2)} ={\frac {\int \Pr(\gamma _{1}|M_{1})\Pr(D|\gamma _{1},M_{1})\,d\gamma _{1}}{\int \Pr(\gamma _{2}|M_{2})\Pr(D|\gamma _{2},M_{2})\,d\gamma _{2}}}
$$

# Technical Details

techinical details here

# Example

In [ ]:
# Clear the environment
rm(list = ls())

# Define genotypes for 5 individuals at 3 variants
# These represent actual alleles at each position
# For example, Individual 1 has genotypes: CC, CT, AT
genotypes <- c(
    "CC", "CT", "AT", # Individual 1
    "TT", "TT", "AA", # Individual 2
    "CT", "CT", "AA", # Individual 3
    "CC", "TT", "AA", # Individual 4
    "CC", "CC", "TT" # Individual 5
)
# Reshape into a matrix
N <- 5 # number of individuals
M <- 3 # number of variants
geno_matrix <- matrix(genotypes, nrow = N, ncol = M, byrow = TRUE)
rownames(geno_matrix) <- paste("Individual", 1:N)
colnames(geno_matrix) <- paste("Variant", 1:M)

alt_alleles <- c("T", "C", "T")
ref_alleles <- c("C", "T", "A")

# Convert to raw genotype matrix using the additive / dominant / recessive model
Xraw_additive <- matrix(0, nrow = N, ncol = M) # dount number of non-reference alleles

rownames(Xraw_additive) <- rownames(geno_matrix)
colnames(Xraw_additive) <- colnames(geno_matrix)

for (i in 1:N) {
    for (j in 1:M) {
        alleles <- strsplit(geno_matrix[i, j], "")[[1]]
        Xraw_additive[i, j] <- sum(alleles == alt_alleles[j])
    }
}
X <- scale(Xraw_additive, center = TRUE, scale = TRUE)

# assign observed height for the 5 individuals
Y_raw <- c(180, 160, 158, 155, 193)
Y <- scale(Y_raw)

## Fixed effect model

Let's first compare two competing models with fixed effect sizes (same models as we discussed in `likelihood`):

- **Model 1 ($M_1$)**: The variant has no effect on height ($\beta=0$)
- **Model 2 ($M_2$)**: The variant has moderate effect on height ($\beta=0.5$)

For both models, we assume:
- $\mathbf{Y} = \mathbf{X}\beta + \epsilon$, where $\epsilon \sim N(0, \sigma^2)$
- $\sigma^2$ is the residual variance (fixed at 1 for standardized data)


The Bayes Factor ($K_{1,2}$) (comparing model 1 and model 2) is the ratio of the marginal likelihoods of the two models:

$$BF_{1,2} = \frac{p(Y|M_1)}{p(Y|M_2)}$$

For models with **fixed effect sizes**, the marginal likelihood is simply the likelihood function evaluated at the fixed parameter value:

$$
p(Y|M_1) = p(Y|\beta=0, M_1)\\
p(Y|M_2) = p(Y|\beta=1, M_2)
$$

In [23]:
# Function to calculate the likelihood for a fixed effect model
# assuming normal distribution for residuals
likelihood <- function(beta, sd, X, Y) {
    # beta: effect size parameter
    # sd: standard deviation of residuals
    # X: genotype values
    # Y: observed phenotype values

    # Calculate expected values under the model
    mu <- X * beta

    # Calculate likelihood (product of normal densities)
    prod(dnorm(Y, mean = mu, sd = sd, log = FALSE))
}

# Function to calculate Bayes Factor between two fixed effect models
bayes_factor <- function(X, Y, beta1, beta2, sigma = 1) {
    # Calculate likelihoods
    likelihood_M1 <- likelihood(beta1, sigma, X, Y)
    likelihood_M2 <- likelihood(beta2, sigma, X, Y)
    
    # Calculate Bayes Factor (M1 vs M2)
    BF12 <- likelihood_M1 / likelihood_M2
    return(BF12)
}


In [24]:
# Initialize results table
bf_table <- data.frame(
    Variant = paste("Variant", 1:M),
    Likelihood_M1 = numeric(M), # Likelihood for Model 1 (β=0)
    Likelihood_M2 = numeric(M), # Likelihood for Model 3 (β=1)
    BF12 = numeric(M), # Bayes Factor (M3 vs M1)
    log_BF12 = numeric(M) # Log of Bayes Factor
)

# Set parameters for models
sigma <- 1 # Standard deviation of residuals
beta1 <- 0 # Effect size for Model 1
beta2 <- 0.5 # Effect size for Model 2

# Calculate Bayes Factors for each variant
for (j in 1:M) {
    X_j <- X[, j] # Genotypes for variant j

    # Calculate likelihoods
    bf_table[j, "Likelihood_M1"] <- likelihood(beta1, sigma, X_j, Y)
    bf_table[j, "Likelihood_M2"] <- likelihood(beta2, sigma, X_j, Y)

    # Calculate Bayes Factor
    bf_table[j, "BF12"] <- bf_table[j, "Likelihood_M1"] / bf_table[j, "Likelihood_M2"]
    bf_table[j, "log_BF12"] <- log(bf_table[j, "BF12"])
}
bf_table

Variant,Likelihood_M1,Likelihood_M2,BF12,log_BF12
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Variant 1,0.001367607,0.0003050987,4.4825078,1.500183
Variant 2,0.001367607,0.0045633971,0.2996906,-1.205005
Variant 3,0.001367607,0.0059679052,0.2291603,-1.473333


Note that under the two full specific models, the Bayes factor is exactly the likelihood ratio that we get from `likelihood ratio` notebook [FIXME with link].

## Random effect model

---
title: "Bayes Factor Calculation for Random Effect Models"
output: html_document
---

```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = TRUE)
```

## Random Effect Models for Genetic Association

This document illustrates how to calculate the Bayes factor between two random effect models (Model 4 and Model 5) to evaluate genetic association with height.

## Model Specification

For our Bayes Factor analysis, we'll compare two random effect models:

- **Model 4**: The effect size follows a normal distribution with smaller variance
  - β ~ N(0, 0.1²)
  - Y = X·β + ε, where ε ~ N(0, σ²)

- **Model 5**: The effect size follows a normal distribution with larger variance  
  - β ~ N(0, 0.5²)
  - Y = X·β + ε, where ε ~ N(0, σ²)

In both models:
- β is the random effect of the genetic variant on height
- X is the genotype data (standardized)
- Y is the observed phenotype (standardized height)
- ε is the residual error term (σ² = 1)

## Implementation

First we'll assume the data preparation has already been done:

```{r random_effects_models}
# Assuming X and Y are already standardized as in the previous code

# Function to calculate the marginal likelihood for a random effect model
# This involves integrating out the random effect parameter β
random_effect_marginal_likelihood <- function(X, Y, prior_var, residual_var = 1) {
  # For random effect models, the marginal likelihood has a closed form
  # when using normal priors and normal likelihood
  n <- length(Y)
  
  # Calculate posterior variance
  post_var <- 1 / (1/prior_var + sum(X^2)/residual_var)
  
  # Calculate posterior mean
  post_mean <- post_var * sum(X * Y) / residual_var
  
  # Calculate marginal likelihood
  log_ml <- -n/2 * log(2 * pi * residual_var) - 
            1/2 * log(1 + prior_var * sum(X^2) / residual_var) -
            1/(2 * residual_var) * (sum(Y^2) - post_mean^2/post_var)
  
  return(exp(log_ml))
}

# Function to calculate Bayes Factor between two random effect models
bayes_factor_random <- function(X, Y, prior_var1, prior_var2, residual_var = 1) {
  # Calculate marginal likelihoods
  ml_model4 <- random_effect_marginal_likelihood(X, Y, prior_var1, residual_var)
  ml_model5 <- random_effect_marginal_likelihood(X, Y, prior_var2, residual_var)
  
  # Calculate Bayes Factor (Model 4 vs Model 5)
  BF45 <- ml_model4 / ml_model5
  
  return(BF45)
}

# Define model parameters
prior_var_model4 <- 0.1^2  # Smaller variance for Model 4
prior_var_model5 <- 0.5^2  # Larger variance for Model 5
residual_var <- 1          # Residual variance (fixed)

# Initialize results table
bf_random_table <- data.frame(
  Variant = paste("Variant", 1:M),
  ML_Model4 = numeric(M),    # Marginal likelihood for Model 4
  ML_Model5 = numeric(M),    # Marginal likelihood for Model 5
  BF45 = numeric(M),         # Bayes Factor (Model 4 vs Model 5)
  log_BF45 = numeric(M)      # Log of Bayes Factor
)

# Calculate Bayes Factors for each variant
for (j in 1:M) {
  X_j <- X[, j]  # Genotypes for variant j
  
  # Calculate marginal likelihoods
  bf_random_table[j, "ML_Model4"] <- random_effect_marginal_likelihood(
    X_j, Y, prior_var_model4, residual_var
  )
  bf_random_table[j, "ML_Model5"] <- random_effect_marginal_likelihood(
    X_j, Y, prior_var_model5, residual_var
  )
  
  # Calculate Bayes Factor
  bf_random_table[j, "BF45"] <- bf_random_table[j, "ML_Model4"] / 
                               bf_random_table[j, "ML_Model5"]
  bf_random_table[j, "log_BF45"] <- log(bf_random_table[j, "BF45"])
}

# Add interpretation column based on Jeffrey's scale
bf_random_table$Evidence <- cut(
  bf_random_table$BF45, 
  breaks = c(0, 1/100, 1/10, 1/3, 1, 3, 10, 30, 100, Inf),
  labels = c("Extreme for M5", "Very strong for M5", "Strong for M5", 
             "Moderate for M5", "Not worth mentioning", 
             "Moderate for M4", "Strong for M4", 
             "Very strong for M4", "Extreme for M4"),
  right = FALSE
)

# Display results
print(bf_random_table)
```

## Explanation of the Approach

In random effect models, the effect size parameter β is not fixed but follows a probability distribution. For Models 4 and 5, we assume β follows a normal distribution with mean 0 and different variances.

To calculate the Bayes factor between two random effect models, we need to:

1. **Calculate the marginal likelihood** for each model by integrating out the random effect:

   $$p(Y|M) = \int p(Y|β,M)p(β|M)dβ$$

   Where:
   - p(Y|β,M) is the likelihood function (normal density in our case)
   - p(β|M) is the prior distribution on the effect size (also normal)
   
2. **Compute the Bayes factor** as the ratio of marginal likelihoods:

   $$BF_{45} = \frac{p(Y|M_4)}{p(Y|M_5)}$$

With normal priors and normal likelihood, this integration has a closed-form solution, which we've implemented in the `random_effect_marginal_likelihood` function.

## Mathematical Derivation

For a linear model Y = Xβ + ε with:
- Y as an n×1 vector of standardized heights
- X as an n×1 vector of standardized genotypes
- β ~ N(0, τ²) as the random effect
- ε ~ N(0, σ²I) as the residual errors

The marginal likelihood p(Y|X,τ²,σ²) is:

$$p(Y|X,τ²,σ²) = \int p(Y|X,β,σ²)p(β|τ²)dβ$$

$$p(Y|X,τ²,σ²) = \frac{1}{(2π)^{n/2}|σ²I+τ²XX'|^{1/2}}exp(-\frac{1}{2}Y'(σ²I+τ²XX')^{-1}Y)$$

Using the matrix inversion lemma and simplifying:

$$p(Y|X,τ²,σ²) = \frac{1}{(2πσ²)^{n/2}(1+τ²X'X/σ²)^{1/2}}exp(-\frac{1}{2σ²}(Y'Y-\frac{τ²(X'Y)²}{σ²+τ²X'X}))$$

This is the formula implemented in our code.

## Visualizing Model Comparisons

```{r visualization}
# Load required libraries
library(ggplot2)
library(gridExtra)

# Plot the prior distributions for effect sizes in both models
beta_range <- seq(-1, 1, length.out = 200)
prior_model4 <- dnorm(beta_range, mean = 0, sd = sqrt(prior_var_model4))
prior_model5 <- dnorm(beta_range, mean = 0, sd = sqrt(prior_var_model5))

prior_data <- data.frame(
  Beta = rep(beta_range, 2),
  Density = c(prior_model4, prior_model5),
  Model = rep(c("Model 4 (narrow prior)", "Model 5 (wide prior)"), each = length(beta_range))
)

prior_plot <- ggplot(prior_data, aes(x = Beta, y = Density, color = Model)) +
  geom_line(size = 1) +
  labs(title = "Prior Distributions for Effect Size",
       x = "Effect Size (β)",
       y = "Density") +
  theme_minimal()

# Plot log Bayes factors
bf_plot <- ggplot(bf_random_table, aes(x = Variant, y = log_BF45)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
  labs(title = "Log Bayes Factors (Model 4 vs Model 5)",
       subtitle = "log(BF45) > 0 favors Model 4 (narrow prior), log(BF45) < 0 favors Model 5 (wide prior)",
       y = "log(BF45)") +
  theme_minimal()

# Arrange plots
grid.arrange(prior_plot, bf_plot, ncol = 1)
```

## Comparison with Fixed Effect Models

We can also calculate the Bayes factors comparing the fixed effect models (Models 1, 2, 3) with the random effect models (Models 4, 5):

```{r compare_fixed_random}
# Define fixed effect parameters
beta1 <- 0    # No effect (Model 1)
beta2 <- 0.5  # Moderate effect (Model 2)
beta3 <- 1    # Strong effect (Model 3)

# Function to calculate likelihood for fixed effect model
fixed_effect_likelihood <- function(X, Y, beta, residual_var = 1) {
  mu <- X * beta
  prod(dnorm(Y, mean = mu, sd = sqrt(residual_var)))
}

# Initialize comparison table
comparison_table <- data.frame(
  Variant = paste("Variant", 1:M),
  L_Model1 = numeric(M),
  L_Model2 = numeric(M),
  L_Model3 = numeric(M),
  ML_Model4 = bf_random_table$ML_Model4,
  ML_Model5 = bf_random_table$ML_Model5
)

# Calculate likelihoods for fixed effect models
for (j in 1:M) {
  X_j <- X[, j]
  
  comparison_table[j, "L_Model1"] <- fixed_effect_likelihood(X_j, Y, beta1)
  comparison_table[j, "L_Model2"] <- fixed_effect_likelihood(X_j, Y, beta2)
  comparison_table[j, "L_Model3"] <- fixed_effect_likelihood(X_j, Y, beta3)
}

# Calculate various Bayes factors
comparison_table$BF14 <- comparison_table$L_Model1 / comparison_table$ML_Model4
comparison_table$BF24 <- comparison_table$L_Model2 / comparison_table$ML_Model4
comparison_table$BF34 <- comparison_table$L_Model3 / comparison_table$ML_Model4
comparison_table$BF15 <- comparison_table$L_Model1 / comparison_table$ML_Model5
comparison_table$BF25 <- comparison_table$L_Model2 / comparison_table$ML_Model5
comparison_table$BF35 <- comparison_table$L_Model3 / comparison_table$ML_Model5

# Display comparison table
print(comparison_table[, c("Variant", "BF14", "BF24", "BF34", "BF15", "BF25", "BF35")])
```

## Discussion

Random effect models offer a more flexible approach than fixed effect models by allowing the effect size to vary according to a probability distribution. This is particularly useful when:

1. We are uncertain about the exact effect size
2. We believe effects may vary across different contexts or populations
3. We want to account for heterogeneity in genetic effects

The Bayes factor between two random effect models (BF45) quantifies the relative evidence for different prior distributions on the effect size. A BF45 > 1 indicates that Model 4 (with narrower prior) is more supported by the data, while BF45 < 1 favors Model 5 (with wider prior).

When comparing fixed and random effect models, we're essentially testing whether the data supports a specific effect size or a distribution of possible effect sizes. These comparisons can help identify the most appropriate modeling approach for genetic association studies.

The key formula for the Bayes factor between two random effect models is:

$$BF_{45} = \frac{p(D|M_4)}{p(D|M_5)} = \frac{\int p(D|\beta,M_4)p(\beta|M_4)d\beta}{\int p(D|\beta,M_5)p(\beta|M_5)d\beta}$$

This approach provides a principled way to incorporate prior knowledge about effect sizes while allowing for uncertainty in genetic association studies.